In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates["custom"] = go.layout.Template(
    layout=go.Layout(
        colorway=px.colors.qualitative.Bold,
    )
)
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from plotly.subplots import make_subplots
from pathlib import Path

import copy

In [2]:
df_dict = {
    "Foundation Model": pd.read_csv(
        "/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/features/foundation_model/task1/test_features.csv"
    ),
    "Supervised": pd.read_csv(
        "/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/features/supervised/task1/test_features.csv"
    ),
    "Med3D": pd.read_csv(
        "/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/features/med3d/task1/test_features.csv"
    ),
    "Models Genesis": pd.read_csv(
        "/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/features/models_genesis/task1/test_features.csv"
    ),
}

In [3]:
feature_methods = {
    "tSNE, perplexity=30": TSNE(n_components=2, perplexity=30, random_state=42),
    "tSNE, perplexity=50": TSNE(n_components=2, perplexity=50, random_state=42),
    "tSNE, perplexity=100": TSNE(n_components=2, perplexity=100, random_state=42),
    "PCA": PCA(n_components=2, random_state=42),
}

In [4]:
lesion_tags = ["bone", "abdomen", "mediastinum", "liver", "lung", "kidney", "soft tissue", "pelvis"]

In [7]:
for model_label, df in df_dict.items():
    for method_label, feature_method in feature_methods.items():
        embedding = pd.DataFrame()
        out = feature_method.fit_transform(df.filter(regex="feature|pred"))
        embedding["feature_0"] = out[:, 0]
        embedding["feature_1"] = out[:, 1]
        embedding["site"] = df["Coarse_lesion_type"].apply(lambda x: lesion_tags[x])

        colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f"]

        # fig1 = px.density_contour(
        #     embedding,
        #     x="feature_0",
        #     y="feature_1",
        #     color="site",
        #     width=600,
        #     height=400,
        #     color_discrete_sequence=colors,
        #     title=" ",
        #     template="simple_white+custom",
        #     labels={"feature_0": "Dimension 1", "feature_1": "Dimension 2"},
        # )
        fig2 = px.scatter(
            embedding,
            color_discrete_sequence=colors,
            x="feature_0",
            y="feature_1",
            color="site",
            title=" ",
            template="simple_white+custom",
            labels={"feature_0": "Dimension 1", "feature_1": "Dimension 2"},
        )
        fig = go.Figure(data=fig1.data + fig2.data, layout=fig2.layout)
        fig.update_layout(width=600, height=500, showlegend=False)
        fig.write_image(f"figures_vector_pdf/feature_rep_{model_label}_{method_label}.pdf")
        fig.show()
        break
    break